In [1]:
"""Bayesian neural network using variational inference
(see, e.g., Blundell et al. (2015); Kucukelbir et al. (2016)).
Inspired by autograd's Bayesian neural network example.
This example prettifies some of the tensor naming for visualization in
TensorBoard. To view TensorBoard, run `tensorboard --logdir=log`.
References
----------
http://edwardlib.org/tutorials/bayesian-neural-network
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import edward as ed
import numpy as np
import tensorflow as tf

from edward.models import Normal

tf.flags.DEFINE_integer("N", 40, "Number of data points.")
tf.flags.DEFINE_integer("D", 1, "Number of features.")

FLAGS = tf.flags.FLAGS


def build_toy_dataset(N=40, noise_std=0.1):
    D = 1
    X = np.concatenate([np.linspace(0, 2, num=N / 2),
                      np.linspace(6, 8, num=N / 2)])
    y = np.cos(X) + np.random.normal(0, noise_std, size=N)
    X = (X - 4.0) / 4.0
    X = X.reshape((N, D))
    return X, y

def neural_network(X):
    h = tf.tanh(tf.matmul(X, W_0) + b_0)
    h = tf.tanh(tf.matmul(h, W_1) + b_1)
    h = tf.matmul(h, W_2) + b_2
    return tf.reshape(h, [-1])
ed.set_seed(42)

def display_draws(x_data, y_data, draw_inputs, draw_outputs, label):
    fig = plt.figure(figsize=(10, 6))
    ax = fig.add_subplot(111)
    ax.plot(x_data, y_data, 'ks', alpha=0.5, label='(x, y)')
    ax.plot(draw_inputs, draw_outputs[0].T, 'r', lw=2, alpha=0.5, label=label)
    ax.plot(draw_inputs, draw_outputs[1:].T, 'r', lw=2, alpha=0.5)
    ax.set_xlim([-5, 5])
    ax.set_ylim([-2, 3])
    ax.legend()
    plt.show()

/home/disij/.local/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [2]:
# DATA
X_train, y_train = build_toy_dataset(FLAGS.N)
# MODEL
with tf.name_scope("model"):
    W_0 = Normal(loc=tf.zeros([FLAGS.D, 10]), scale=tf.ones([FLAGS.D, 10]), name="W_0")
    W_1 = Normal(loc=tf.zeros([10, 10]), scale=tf.ones([10, 10]), name="W_1")
    W_2 = Normal(loc=tf.zeros([10, 1]), scale=tf.ones([10, 1]), name="W_2")
    b_0 = Normal(loc=tf.zeros(10), scale=tf.ones(10), name="b_0")
    b_1 = Normal(loc=tf.zeros(10), scale=tf.ones(10), name="b_1")
    b_2 = Normal(loc=tf.zeros(1), scale=tf.ones(1), name="b_2")
    X = tf.placeholder(tf.float32, [FLAGS.N, FLAGS.D], name="X")
    y = Normal(loc=neural_network(X), scale=0.1 * tf.ones(FLAGS.N), name="y")
# INFERENCE
with tf.variable_scope("posterior"):
    with tf.variable_scope("qW_0"):
        loc = tf.get_variable("loc", [FLAGS.D, 10])
        scale = tf.nn.softplus(tf.get_variable("scale", [FLAGS.D, 10]))
        qW_0 = Normal(loc=loc, scale=scale)
    with tf.variable_scope("qW_1"):
        loc = tf.get_variable("loc", [10, 10])
        scale = tf.nn.softplus(tf.get_variable("scale", [10, 10]))
        qW_1 = Normal(loc=loc, scale=scale)
    with tf.variable_scope("qW_2"):
        loc = tf.get_variable("loc", [10, 1])
        scale = tf.nn.softplus(tf.get_variable("scale", [10, 1]))
        qW_2 = Normal(loc=loc, scale=scale)
    with tf.variable_scope("qb_0"):
        loc = tf.get_variable("loc", [10])
        scale = tf.nn.softplus(tf.get_variable("scale", [10]))
        qb_0 = Normal(loc=loc, scale=scale)
    with tf.variable_scope("qb_1"):
        loc = tf.get_variable("loc", [10])
        scale = tf.nn.softplus(tf.get_variable("scale", [10]))
        qb_1 = Normal(loc=loc, scale=scale)
    with tf.variable_scope("qb_2"):
        loc = tf.get_variable("loc", [1])
        scale = tf.nn.softplus(tf.get_variable("scale", [1]))
        qb_2 = Normal(loc=loc, scale=scale)

/home/disij/.local/lib/python2.7/site-packages/ipykernel/__main__.py:28: DeprecationWarning: object of type <type 'float'> cannot be safely interpreted as an integer.
/home/disij/.local/lib/python2.7/site-packages/ipykernel/__main__.py:29: DeprecationWarning: object of type <type 'float'> cannot be safely interpreted as an integer.


In [3]:
tf.contrib.distributions.kl = tf.contrib.distributions.kl_divergence
inference = ed.KLqp({W_0: qW_0, b_0: qb_0,
                   W_1: qW_1, b_1: qb_1,
                   W_2: qW_2, b_2: qb_2}, data={X: X_train, y: y_train})
inference.run(n_iter=10000, logdir='log')

/home/disij/.local/lib/python2.7/site-packages/edward/util/random_variables.py:48: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  not np.issubdtype(value.dtype, np.float) and \


10000/10000 [100%] ██████████████████████████████ Elapsed: 11s | Loss: 493.365


In [4]:
# Sample neural networks from our variational model.
rs = np.random.RandomState(0)
inputs = np.linspace(-5, 5, num=500, dtype=np.float32)
nx = tf.expand_dims(inputs, 1)
mus = tf.stack(
    [neural_network(nx, qW_0.sample(), qW_1.sample(), qb_0.sample(), qb_1.sample())
     for _ in range(10)])

# Visualize prior draws.
sess = ed.get_session()
tf.global_variables_initializer().run()
outputs = mus.eval()
display_draws(x_train, y_train, inputs, outputs, 'prior draws')

TypeError: neural_network() takes exactly 1 argument (5 given)

In [ ]:
# Minimize the KL-divergence of our posterior approximators.
tf.contrib.distributions.kl = tf.contrib.distributions.kl_divergence
inference = ed.KLqp({W_0: qW_0, b_0: qb_0, W_1: qW_1, b_1: qb_1},
                    data={x: x_train, y: y_train})
inference.run(n_iter=5000)